<a href="https://colab.research.google.com/github/redpineK/deeplearning/blob/master/embedding2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 - https://www.kaggle.com/code/rajmehra03/a-detailed-explanation-of-keras-embedding-layer/notebook

In [5]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:


#nltk
# import nltk

#stop-words
# from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize
#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

In [12]:
sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)

In [13]:
vocab_size=50 
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document",i+1," is : ",one_hot(doc,50))

The encoding for document 1  is :  [47, 35, 26, 47, 44, 19]
The encoding for document 2  is :  [36, 14, 47, 44, 19, 47, 26, 47, 3]
The encoding for document 3  is :  [5, 34, 35, 48, 2, 19, 46, 48, 14, 3, 19, 2]


In [16]:
# length of maximum document. will be nedded whenever create embeddings for the words
maxlen=-1
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  12


In [17]:
# now to create embeddings all of our docs need to be of same length. hence we can pad the docs with zeros.
pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)
print("No of padded documents: ",len(pad_corp))

No of padded documents:  3


In [18]:
for i,doc in enumerate(pad_corp):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [47 35 26 47 44 19  0  0  0  0  0  0]
The padded encoding for document 2  is :  [36 14 47 44 19 47 26 47  3  0  0  0]
The padded encoding for document 3  is :  [ 5 34 35 48  2 19 46 48 14  3 19  2]


In [19]:
# specifying the input shape
input=Input(shape=(no_docs,maxlen),dtype='float64')

In [20]:
'''
shape of input. 
each document has 12 element or words which is the value of our maxlen variable.

'''
word_input=Input(shape=(maxlen,),dtype='float64')  

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)

word_vec=Flatten()(word_embedding) # flatten
embed_model =Model([word_input],word_vec) # combining all into a Keras model

In [21]:
embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc']) 
# compiling the model. parameters can be tuned as always.

In [22]:
print(type(word_embedding))
print(word_embedding)

<class 'keras.engine.keras_tensor.KerasTensor'>
KerasTensor(type_spec=TensorSpec(shape=(None, 12, 8), dtype=tf.float32, name=None), name='embedding/embedding_lookup/Identity_1:0', description="created by layer 'embedding'")


In [23]:
print(embed_model.summary()) # summary of the model

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12)]              0         
                                                                 
 embedding (Embedding)       (None, 12, 8)             400       
                                                                 
 flatten (Flatten)           (None, 96)                0         
                                                                 
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

1/1 [==============================] - 0s 187ms/step


In [25]:
print("Shape of embeddings : ",embeddings.shape)
print(embeddings)

Shape of embeddings :  (3, 96)
[[ 0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467 -0.01063299
  -0.04568898 -0.02832687  0.00406151 -0.04067155 -0.0216952  -0.03526106
   0.0099689  -0.01227249 -0.01924158 -0.03416069 -0.00846242 -0.02669038
  -0.03737707 -0.03100523  0.01753208 -0.00716165 -0.03958933  0.02342893
   0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467 -0.01063299
  -0.04568898 -0.02832687 -0.03213499  0.00381348 -0.0323016  -0.02118363
   0.03843411 -0.01977954 -0.0376313   0.00078212 -0.04762596 -0.03199545
  -0.00610454  0.00913912  0.04712454  0.0447777   0.03903276  0.04508351
   0.00998991 -0.0211266   0.01373998  0.00513347  0.02413077  0.0463694
   0.04576981  0.02851266  0.00998991 -0.0211266   0.01373998  0.00513347
   0.02413077  0.0463694   0.04576981  0.02851266  0.00998991 -0.0211266
   0.01373998  0.00513347  0.02413077  0.0463694   0.04576981  0.02851266
   0.00998991 -0.0211266   0.01373998  0.00513347  0.02413077  0.0463694
   0.04576

In [26]:
embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

Shape of embeddings :  (3, 12, 8)
[[[ 0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467
   -0.01063299 -0.04568898 -0.02832687]
  [ 0.00406151 -0.04067155 -0.0216952  -0.03526106  0.0099689
   -0.01227249 -0.01924158 -0.03416069]
  [-0.00846242 -0.02669038 -0.03737707 -0.03100523  0.01753208
   -0.00716165 -0.03958933  0.02342893]
  [ 0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467
   -0.01063299 -0.04568898 -0.02832687]
  [-0.03213499  0.00381348 -0.0323016  -0.02118363  0.03843411
   -0.01977954 -0.0376313   0.00078212]
  [-0.04762596 -0.03199545 -0.00610454  0.00913912  0.04712454
    0.0447777   0.03903276  0.04508351]
  [ 0.00998991 -0.0211266   0.01373998  0.00513347  0.02413077
    0.0463694   0.04576981  0.02851266]
  [ 0.00998991 -0.0211266   0.01373998  0.00513347  0.02413077
    0.0463694   0.04576981  0.02851266]
  [ 0.00998991 -0.0211266   0.01373998  0.00513347  0.02413077
    0.0463694   0.04576981  0.02851266]
  [ 0.00998991 -0.0211266   0.01373998  

In [27]:
for i,doc in enumerate(embeddings):
    for j,word in enumerate(doc):
        print("The encoding for ",j+1,"th word","in",i+1,"th document is : \n\n",word)

The encoding for  1 th word in 1 th document is : 

 [ 0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467 -0.01063299
 -0.04568898 -0.02832687]
The encoding for  2 th word in 1 th document is : 

 [ 0.00406151 -0.04067155 -0.0216952  -0.03526106  0.0099689  -0.01227249
 -0.01924158 -0.03416069]
The encoding for  3 th word in 1 th document is : 

 [-0.00846242 -0.02669038 -0.03737707 -0.03100523  0.01753208 -0.00716165
 -0.03958933  0.02342893]
The encoding for  4 th word in 1 th document is : 

 [ 0.00021515  0.04049387 -0.04500476 -0.00864762 -0.02606467 -0.01063299
 -0.04568898 -0.02832687]
The encoding for  5 th word in 1 th document is : 

 [-0.03213499  0.00381348 -0.0323016  -0.02118363  0.03843411 -0.01977954
 -0.0376313   0.00078212]
The encoding for  6 th word in 1 th document is : 

 [-0.04762596 -0.03199545 -0.00610454  0.00913912  0.04712454  0.0447777
  0.03903276  0.04508351]
The encoding for  7 th word in 1 th document is : 

 [ 0.00998991 -0.0211266   0.01373998